In [1]:
!pip install protobuf==3.20.0
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [2]:
!pip install textstat

In [3]:
from flask import Flask, request, jsonify
import os
import ktrain
from ktrain import text

model_path = '../bert_model'
model = ktrain.load_predictor(model_path)

def get_cefr_level(text, model):
    prediction = model.predict(text)

    # Map the prediction to CEFR levels (customize this mapping as needed)
    cefr_levels = {
        'label_0':'Elementary',
        'label_1':'Intermediate',
        'label_2':'Advanced'
                }

    # Return the CEFR level based on the prediction
    return cefr_levels[prediction]

input_text = "This is a sample text for classification."

cefr_level = get_cefr_level(input_text, model)

print(f"CEFR Level: {cefr_level}")

/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


CEFR Level: Elementary


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from gcs_funcs import list_blobs, list_blobs_pd

BUCKET_NAME = "kisai-data-msca310019-capstone"
FULL_BUCKET_NAME = "gs://" + BUCKET_NAME 
FOLDER_NAME = "Text_Simplification"

files_df = list_blobs_pd(bucket_name=BUCKET_NAME, 
           folder_name=FOLDER_NAME)
file_names = list(files_df.iloc[1:]["Name"])

file_names

['Text_Simplification/BreakingNewsLevels.csv',
 'Text_Simplification/FinalBreakingNewsLevels.csv',
 'Text_Simplification/NewsInLevels_dataset_TextSimplification.csv',
 'Text_Simplification/dolly_df.csv',
 'Text_Simplification/finetuned_llama2_responses.csv',
 'Text_Simplification/flant5_df.csv',
 'Text_Simplification/huggingface_cefr_labeled.csv',
 'Text_Simplification/huggingface_cochrane_dataset.csv',
 'Text_Simplification/raw_data.csv',
 'Text_Simplification/simplified_df.csv',
 'Text_Simplification/simplified_df_cefr_labeled.csv',
 'Text_Simplification/simplified_df_new.csv',
 'Text_Simplification/trial_smpl.csv',
 'Text_Simplification/trial_smpl_large.csv',
 'Text_Simplification/trial_smpl_medium.csv',
 'Text_Simplification/wiki_cefr_labeled.csv',
 'Text_Simplification/wiki_large.csv',
 'Text_Simplification/wiki_small.csv']

In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv(FULL_BUCKET_NAME + '/Text_Simplification/finetuned_llama2_responses.csv')

In [30]:
list(df)

['instruction',
 'context',
 'response',
 'model_response',
 'oob_model_response',
 'model_v2_response']

In [32]:
df = df.iloc[:, :-1]
list(df)

['instruction', 'context', 'response', 'model_response', 'oob_model_response']

In [33]:
import textstat

In [34]:
df_new = df.dropna(axis=0)

pd.options.display.max_colwidth = None
df_new.tail(1)

,instruction,context,response,model_response,oob_model_response
1020,Simplify the following context from CEFR Level 2 to CEFR Level 1,"People are angry because a video of the killings in Christchurch is still on social media. A terrorist shot and killed 50 worshippers at two mosques. He live-streamed the shootings on Facebook. Many people think social media sites did not do enough to stop the video spreading. Social media users asked people to stop sharing it. One said: ""That is what the terrorist wanted."" Another said sites quickly take down music videos but not terror videos.","People are angry. A video of the killings in Christchurch is still on social media. A terrorist killed 50 people at two mosques. He put a live-stream on Facebook. People say social media did stop the video spreading. They asked people to stop sharing it. One said: ""That is what the terrorist wanted.""","\nPeople are angry because a video of the killings in Christchurch is still on social media. A terrorist shot and killed 50 worshippers at two mosques. He live-streamed the shootings on Facebook. Many people think social media sites did not do enough to stop the video spreading. Social media users asked people to stop sharing it. One said: ""That is what the terrorist wanted."" Another said sites quickly take down music videos but not terror videos.\n\n### End: \n\n### End: \n\n### Video:\n\nInput:\n\n<https://www.youtube.com/watch?v=gWjn8CxbH","\nPeople are angry because a video of the killings in Christchurch is still on social media. A terrorist shot and killed 50 worshippers at two mosques. He live-streamed the shootings on Facebook. Many people think social media sites did not do enough to stop the video spreading. Social media users asked people to stop sharing it. One said: ""That is what the terrorist wanted."" Another said sites quickly take down music videos but not terror videos.\n\n### Instruction: \nSimplify the following context from CEFR Level 2 to CEFR Level 1\n\n### Input: \nThe president of the United States said he would close the border with"


In [35]:
df_new['instruction'].value_counts()

instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    381
Simplify the following context from CEFR Level 3 to CEFR Level 1    320
Simplify the following context from CEFR Level 3 to CEFR Level 2    320
Name: count, dtype: int64

In [36]:
import re
pattern = r"^(.*?)\n\n.*"

df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL)) 
df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL)) 

/var/tmp/ipykernel_22407/1826758872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
/var/tmp/ipykernel_22407/1826758872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))


In [37]:
df_new.head(2)

,instruction,context,response,model_response,oob_model_response
0,Simplify the following context from CEFR Level 2 to CEFR Level 1,"Donald Trump is interested in buying Greenland. He said it would be like buying property and was, ""a large real estate deal"". He added: ""They've got a lot of valuable minerals."" Mr Trump said: ""Denmark owns it....We protect Denmark....So the concept came up and I said, 'Certainly I'd be [interested].' He said buying Greenland was not his top priority at the moment. He said: ""It's not number one on the burner.""","Donald Trump is interested in buying Greenland. He said it would be, ""a large real estate deal"". He added: ""They've got a lot of valuable minerals."" Mr Trump said: ""Certainly I'd be [interested].' He said buying Greenland was not his top priority. He said: ""It's not number one on the burner.""","\nDonald Trump is interested in buying Greenland. He said it would be like buying property. He said: ""They've got a lot of valuable minerals."" Mr Trump said: ""Denmark owns it. We protect Denmark."" He said buying Greenland was not his top priority. He said: ""It's not number one on the burner.""","\nDonald Trump is interested in buying Greenland. He said it would be like buying property and was, ""a large real estate deal"". He added: ""They've got a lot of valuable minerals."" Mr Trump said: ""Denmark owns it....We protect Denmark....So the concept came up and I said, 'Certainly I'd be [interested].' He said buying Greenland was not his top priority at the moment. He said: ""It's not number one on the burner."""
1,Simplify the following context from CEFR Level 2 to CEFR Level 1,Everyone knows that children don't like eating greens. Parents can't get their children to eat vegetables. A new study explains why. The study looked at the eating habits of young children. The study found that genes can make children dislike greens. The children can become fussy eaters. This means parents do not have to stress if their children do not eat vegetables. Many parents worry about what their children eat.,Everyone knows children don't like eating greens. A new study explains why. The study looked at what young children ate. The study found that genes made children dislike greens. The children also disliked other things. This means parents do not have to stress if their children do not eat vegetables.,\nChildren don't like eating greens. Parents can't get their children to eat vegetables. A study found that genes make children dislike greens. The children can become fussy eaters. This means parents do not have to stress if their children do not eat vegetables.,\nEveryone knows that children don't like eating greens. Parents can't get their children to eat vegetables. A new study explains why. The study looked at the eating habits of young children. The study found that genes can make children dislike greens. The children can become fussy eaters. This means parents do not have to stress if their children do not eat vegetables. Many parents worry about what their children eat.


In [38]:
def calc_smog(text):
    return textstat.smog_index(text)

def calc_flesch(text):
    return textstat.flesch_reading_ease(text)

def calc_dale(text):
    return textstat.dale_chall_readability_score(text)

df_new['response_smog'] = round(df_new['response'].apply(calc_smog),2)
df_new['model_response_smog'] = round(df_new['model_response'].apply(calc_smog),2)
df_new['oob_model_response_smog'] = round(df_new['oob_model_response'].apply(calc_smog),2)

df_new['response_fl'] = round(df_new['response'].apply(calc_flesch),2)
df_new['model_response_fl'] = round(df_new['model_response'].apply(calc_flesch),2)
df_new['oob_model_response_fl'] = round(df_new['oob_model_response'].apply(calc_flesch),2)

df_new['response_dale'] = round(df_new['response'].apply(calc_dale),2)
df_new['model_response_dale'] = round(df_new['model_response'].apply(calc_dale),2)
df_new['oob_model_response_dale'] = round(df_new['oob_model_response'].apply(calc_dale),2)

df_new['response_CEFR'] = df_new['response'].apply(lambda x: get_cefr_level(x, model))
df_new['model_response_CEFR'] = df_new['model_response'].apply(lambda x: get_cefr_level(x, model))
df_new['oob_model_response_CEFR'] = df_new['oob_model_response'].apply(lambda x: get_cefr_level(x, model))

## Train CEFR level
df_new['train_CEFR'] = df_new['context'].apply(lambda x: get_cefr_level(x, model))



/var/tmp/ipykernel_22407/3547480008.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['response_smog'] = round(df_new['response'].apply(calc_smog),2)
/var/tmp/ipykernel_22407/3547480008.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['model_response_smog'] = round(df_new['model_response'].apply(calc_smog),2)
/var/tmp/ipykernel_22407/3547480008.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [50]:
#df_new.head(1)

## Prayut added this

### CEFR Evaluation

#### Ground truth - only CEFR based NO instruct 

In [53]:
df_cefr = df_new.copy()

def map_cefr_to_level(value):
    if value == 'Intermediate':
        return 2
    elif value == 'Advanced':
        return 3
    elif value == 'Elementary':
        return 1
    else:
        return 0


df_cefr = df_cefr.rename(columns={'train_CEFR':'ground_truth_context','response_CEFR':'ground_truth_target'})

# df_cefr['ground_truth_context'] = df_cefr['train_cefr'].apply(map_cefr_to_truth)
# df_cefr['ground_truth_target'] = df_cefr['response_cefr'].apply(map_cefr_to_truth)


df_cefr['result_llm'] = (df_cefr['ground_truth_target'] == df_cefr['model_response_CEFR']).astype(int)
df_cefr['result_oob'] = (df_cefr['ground_truth_target'] == df_cefr['oob_model_response_CEFR']).astype(int)


df_cefr = df_cefr[['ground_truth_context','ground_truth_target','result_llm','result_oob']]
# df_cefr.head()
df_cefr.groupby(['ground_truth_context', 'ground_truth_target'])[['result_llm','result_oob']].mean().reset_index()

,ground_truth_context,ground_truth_target,result_llm,result_oob
0,Advanced,Elementary,0.000000,0.250000
1,Advanced,Intermediate,0.600000,0.166667
2,Elementary,Elementary,0.903846,0.769231
3,Elementary,Intermediate,0.222222,0.222222
4,Intermediate,Elementary,0.214286,0.285714
5,Intermediate,Intermediate,0.914573,0.756281


#### Ground truth - instruct based

In [54]:
df_cefr_1 = df_new.copy()

# Define a mapping function
def map_instruct_level(level):
    if 'Level 1' in level:
        return 'Elementary'
    elif 'Level 2' in level:
        return 'Intermediate'
    elif 'Level 3' in level:
        return 'Advanced'
    else:
        return 'Unknown'

df_cefr_1['ground_truth_context'] = df_cefr_1['instruction'].apply(lambda x: map_instruct_level(x.split(' to ')[0]))
df_cefr_1['ground_truth_target'] = df_cefr_1['instruction'].apply(lambda x: map_instruct_level(x.split(' to ')[1]))

df_cefr_1['result_llm'] = (df_cefr_1['ground_truth_target'] == df_cefr_1['model_response_CEFR']).astype(int)
df_cefr_1['result_oob'] = (df_cefr_1['ground_truth_target'] == df_cefr_1['oob_model_response_CEFR']).astype(int)


df_cefr_1 = df_cefr_1[['ground_truth_context','ground_truth_target','result_llm','result_oob']]
# df_cefr.head()
df_cefr_1.groupby(['ground_truth_context', 'ground_truth_target'])[['result_llm','result_oob']].mean().reset_index()

,ground_truth_context,ground_truth_target,result_llm,result_oob
0,Advanced,Elementary,0.137500,0.259375
1,Advanced,Intermediate,0.868750,0.725000
2,Intermediate,Elementary,0.167979,0.304462


## NEW eval_cefr()  

In [60]:
import re

def map_cefr_to_level(value):
    if value == 'Intermediate':
        return 2
    elif value == 'Advanced':
        return 3
    elif value == 'Elementary':
        return 1
    else:
        return 0
    
def map_instruct_level(level):
    if 'Level 1' in level:
        return 'Elementary'
    elif 'Level 2' in level:
        return 'Intermediate'
    elif 'Level 3' in level:
        return 'Advanced'
    else:
        return 'Unknown'

def eval_cefr(df):
    df_new = df.dropna(axis=0)
    pattern = r"^(.*?)\n\n.*"

    df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL)) 
    df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
    
    df_new['response_CEFR'] = df_new['response'].apply(lambda x: get_cefr_level(x, model))
    df_new['model_response_CEFR'] = df_new['model_response'].apply(lambda x: get_cefr_level(x, model))
    df_new['oob_model_response_CEFR'] = df_new['oob_model_response'].apply(lambda x: get_cefr_level(x, model))

    df_new['train_CEFR'] = df_new['context'].apply(lambda x: get_cefr_level(x, model))
    
    
    df_cefr_no_instruct = df_new.copy()
    df_cefr_no_instruct = df_cefr_no_instruct.rename(columns={'train_CEFR':'ground_truth_context','response_CEFR':'ground_truth_target'})

    df_cefr_no_instruct['result_llm'] = (df_cefr_no_instruct['ground_truth_target'] == df_cefr_no_instruct['model_response_CEFR']).astype(int)
    df_cefr_no_instruct['result_oob'] = (df_cefr_no_instruct['ground_truth_target'] == df_cefr_no_instruct['oob_model_response_CEFR']).astype(int)


    df_cefr_no_instruct = df_cefr_no_instruct[['ground_truth_context','ground_truth_target','result_llm','result_oob']]
    df_cefr_no_instruct = df_cefr_no_instruct.groupby(['ground_truth_context', 'ground_truth_target'])[['result_llm','result_oob']].mean().reset_index()
    
    df_cefr_instruct = df_new.copy()
    
    df_cefr_instruct['ground_truth_context'] = df_cefr_instruct['instruction'].apply(lambda x: map_instruct_level(x.split(' to ')[0]))
    df_cefr_instruct['ground_truth_target'] = df_cefr_instruct['instruction'].apply(lambda x: map_instruct_level(x.split(' to ')[1]))

    df_cefr_instruct['result_llm'] = (df_cefr_instruct['ground_truth_target'] == df_cefr_instruct['model_response_CEFR']).astype(int)
    df_cefr_instruct['result_oob'] = (df_cefr_instruct['ground_truth_target'] == df_cefr_instruct['oob_model_response_CEFR']).astype(int)

    df_cefr_instruct = df_cefr_instruct[['ground_truth_context','ground_truth_target','result_llm','result_oob']]
    df_cefr_instruct = df_cefr_instruct.groupby(['ground_truth_context', 'ground_truth_target'])[['result_llm','result_oob']].mean().reset_index()
    
    return df_cefr_no_instruct, df_cefr_instruct
    

In [61]:
eval_cefr(df=df)

/var/tmp/ipykernel_22407/4053764199.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
/var/tmp/ipykernel_22407/4053764199.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
/var/tmp/ipykernel_22407/4053764199.py:30: SettingWithCopyWarning: 
A value is trying to be set on a c

(  ground_truth_context ground_truth_target  result_llm  result_oob
 0             Advanced          Elementary    0.000000    0.250000
 1             Advanced        Intermediate    0.600000    0.166667
 2           Elementary          Elementary    0.903846    0.769231
 3           Elementary        Intermediate    0.222222    0.222222
 4         Intermediate          Elementary    0.214286    0.285714
 5         Intermediate        Intermediate    0.914573    0.756281,
   ground_truth_context ground_truth_target  result_llm  result_oob
 0             Advanced          Elementary    0.137500    0.259375
 1             Advanced        Intermediate    0.868750    0.725000
 2         Intermediate          Elementary    0.167979    0.304462)

## NEW textstat_eval() 

In [58]:
import textstat

def textstat_eval(df):
    df_new = df.dropna(axis=0)
    pattern = r"^(.*?)\n\n.*"

    df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL)) 
    df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
    
    df_new['response_smog'] = round(df_new['response'].apply(calc_smog),2)
    df_new['model_response_smog'] = round(df_new['model_response'].apply(calc_smog),2)
    df_new['oob_model_response_smog'] = round(df_new['oob_model_response'].apply(calc_smog),2)

    df_new['response_fl'] = round(df_new['response'].apply(calc_flesch),2)
    df_new['model_response_fl'] = round(df_new['model_response'].apply(calc_flesch),2)
    df_new['oob_model_response_fl'] = round(df_new['oob_model_response'].apply(calc_flesch),2)

    df_new['response_dale'] = round(df_new['response'].apply(calc_dale),2)
    df_new['model_response_dale'] = round(df_new['model_response'].apply(calc_dale),2)
    df_new['oob_model_response_dale'] = round(df_new['oob_model_response'].apply(calc_dale),2)
    
    stats = df_new.groupby('instruction')[['response_smog', 'model_response_smog', 'oob_model_response_smog',
                'response_fl', 'model_response_fl','oob_model_response_fl',
                'response_dale', 'model_response_dale','oob_model_response_dale']].describe()
    
    stats = stats[[('response_smog', 'mean'),('response_smog', 'std'),('model_response_smog', 'mean'),('model_response_smog', 'std'),
                   ('oob_model_response_smog', 'mean'), ('oob_model_response_smog', 'std'),
                   ('response_fl', 'mean'),('response_fl', 'std'),('model_response_fl', 'mean'),('model_response_fl', 'std'),
                   ('oob_model_response_fl', 'mean'),('oob_model_response_fl', 'std'),
                   ('response_dale', 'mean'),('response_dale', 'std'), ('model_response_dale', 'mean'),('model_response_dale', 'std'),]]
    
    return stats

In [59]:
textstat_eval(df=df)

/var/tmp/ipykernel_22407/691561938.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['model_response'] = df_new['model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
/var/tmp/ipykernel_22407/691561938.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['oob_model_response'] = df_new['oob_model_response'].apply(lambda x: re.sub(pattern, r'\1', x, flags=re.DOTALL))
/var/tmp/ipykernel_22407/691561938.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy o

response_smog  \
                                                                          mean   
instruction                                                                      
Simplify the following context from CEFR Level 2 to CEFR Level 1      7.315748   
Simplify the following context from CEFR Level 3 to CEFR Level 1      7.412500   
Simplify the following context from CEFR Level 3 to CEFR Level 2      8.170000   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  1.561060   
Simplify the following context from CEFR Level 3 to CEFR Level 1  1.613064   
Simplify the following context from CEFR Level 3 to CEFR Level 2  1.273053   

                                                                 model_response_smog  \
                                                                                mean   
instruction                                                                            
Simplify the following context from CEFR Level 2 to CEFR Level 1            7.759318   
Simplify the following context from CEFR Level 3 to CEFR Level 1            8.382500   
Simplify the following context from CEFR Level 3 to CEFR Level 2            8.505312   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  1.817256   
Simplify the following context from CEFR Level 3 to CEFR Level 1  2.185562   
Simplify the following context from CEFR Level 3 to CEFR Level 2  1.812652   

                                                                 response_fl  \
                                                                        mean   
instruction                                                                    
Simplify the following context from CEFR Level 2 to CEFR Level 1   84.013622   
Simplify the following context from CEFR Level 3 to CEFR Level 1   82.384875   
Simplify the following context from CEFR Level 3 to CEFR Level 2   80.650000   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  8.749001   
Simplify the following context from CEFR Level 3 to CEFR Level 1  7.954910   
Simplify the following context from CEFR Level 3 to CEFR Level 2  7.844883   

                                                                 model_response_fl  \
                                                                              mean   
instruction                                                                          
Simplify the following context from CEFR Level 2 to CEFR Level 1         81.643727   
Simplify the following context from CEFR Level 3 to CEFR Level 1         77.594250   
Simplify the following context from CEFR Level 3 to CEFR Level 2         77.294187   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  9.851194   
Simplify the following context from CEFR Level 3 to CEFR Level 1  9.337990   
Simplify the following context from CEFR Level 3 to CEFR Level 2  9.363398   

                                                                 response_dale  \
                                                                          mean   
instr

In [65]:
stats = df_new.groupby('instruction')[['response_smog', 'model_response_smog', 'oob_model_response_smog',
                'response_fl', 'model_response_fl','oob_model_response_fl',
                'response_dale', 'model_response_dale','oob_model_response_dale']].describe()

list(stats)

[('response_smog', 'count'),
 ('response_smog', 'mean'),
 ('response_smog', 'std'),
 ('response_smog', 'min'),
 ('response_smog', '25%'),
 ('response_smog', '50%'),
 ('response_smog', '75%'),
 ('response_smog', 'max'),
 ('model_response_smog', 'count'),
 ('model_response_smog', 'mean'),
 ('model_response_smog', 'std'),
 ('model_response_smog', 'min'),
 ('model_response_smog', '25%'),
 ('model_response_smog', '50%'),
 ('model_response_smog', '75%'),
 ('model_response_smog', 'max'),
 ('oob_model_response_smog', 'count'),
 ('oob_model_response_smog', 'mean'),
 ('oob_model_response_smog', 'std'),
 ('oob_model_response_smog', 'min'),
 ('oob_model_response_smog', '25%'),
 ('oob_model_response_smog', '50%'),
 ('oob_model_response_smog', '75%'),
 ('oob_model_response_smog', 'max'),
 ('response_fl', 'count'),
 ('response_fl', 'mean'),
 ('response_fl', 'std'),
 ('response_fl', 'min'),
 ('response_fl', '25%'),
 ('response_fl', '50%'),
 ('response_fl', '75%'),
 ('response_fl', 'max'),
 ('model_res

In [40]:
stats = df_new.groupby('instruction')[['response_smog', 'model_response_smog', 
                'response_fl', 'model_response_fl',
                'response_dale', 'model_response_dale']].describe()


In [62]:
stats

response_smog  \
                                                                         count   
instruction                                                                      
Simplify the following context from CEFR Level 2 to CEFR Level 1         381.0   
Simplify the following context from CEFR Level 3 to CEFR Level 1         320.0   
Simplify the following context from CEFR Level 3 to CEFR Level 2         320.0   

                                                                            \
                                                                      mean   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  7.315748   
Simplify the following context from CEFR Level 3 to CEFR Level 1  7.412500   
Simplify the following context from CEFR Level 3 to CEFR Level 2  8.170000   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  1.561060   
Simplify the following context from CEFR Level 3 to CEFR Level 1  1.613064   
Simplify the following context from CEFR Level 3 to CEFR Level 2  1.273053   

                                                                            \
                                                                  min  25%   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  3.1  6.3   
Simplify the following context from CEFR Level 3 to CEFR Level 1  3.1  6.4   
Simplify the following context from CEFR Level 3 to CEFR Level 2  5.1  7.4   

                                                                            \
                                                                  50%  75%   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  7.2  8.8   
Simplify the following context from CEFR Level 3 to CEFR Level 1  7.6  8.8   
Simplify the following context from CEFR Level 3 to CEFR Level 2  8.1  8.8   

                                                                        \
                                                                   max   
instruction                                                              
Simplify the following context from CEFR Level 2 to CEFR Level 1  10.7   
Simplify the following context from CEFR Level 3 to CEFR Level 1  10.7   
Simplify the following context from CEFR Level 3 to CEFR Level 2  11.6   

                                                                 model_response_smog  \
                                                                               count   
instruction                                                                            
Simplify the following context from CEFR Level 2 to CEFR Level 1               381.0   
Simplify the following context from CEFR Level 3 to CEFR Level 1               320.0   
Simplify the following context from CEFR Level 3 to CEFR Level 2               320.0   

                                                                            \
                                                                      mean   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  7.759318   
Simplify the following context from CEFR Level 3 to CEFR Level 1  8.382500   
Simplify the following context from CEFR Level 3 to CEFR Level 2  8.505312   

                                                                  ...  \
                                                                  ...   
instruction                                                       ...   
Simplify the following context fro

In [48]:
stats[[('response_smog', 'mean'),('response_smog', 'std'),('model_response_smog', 'mean'),('model_response_smog', 'std'),
       ('response_fl', 'mean'),
       ('response_fl', 'std'),('model_response_fl', 'mean'),
       ('model_response_fl', 'std'),('response_dale', 'mean'),
       ('response_dale', 'std'), ('model_response_dale', 'mean'),
       ('model_response_dale', 'std')]]

response_smog  \
                                                                          mean   
instruction                                                                      
Simplify the following context from CEFR Level 2 to CEFR Level 1      7.315748   
Simplify the following context from CEFR Level 3 to CEFR Level 1      7.412500   
Simplify the following context from CEFR Level 3 to CEFR Level 2      8.170000   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  1.561060   
Simplify the following context from CEFR Level 3 to CEFR Level 1  1.613064   
Simplify the following context from CEFR Level 3 to CEFR Level 2  1.273053   

                                                                 model_response_smog  \
                                                                                mean   
instruction                                                                            
Simplify the following context from CEFR Level 2 to CEFR Level 1            7.759318   
Simplify the following context from CEFR Level 3 to CEFR Level 1            8.382500   
Simplify the following context from CEFR Level 3 to CEFR Level 2            8.505312   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  1.817256   
Simplify the following context from CEFR Level 3 to CEFR Level 1  2.185562   
Simplify the following context from CEFR Level 3 to CEFR Level 2  1.812652   

                                                                 response_fl  \
                                                                        mean   
instruction                                                                    
Simplify the following context from CEFR Level 2 to CEFR Level 1   84.013622   
Simplify the following context from CEFR Level 3 to CEFR Level 1   82.384875   
Simplify the following context from CEFR Level 3 to CEFR Level 2   80.650000   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  8.749001   
Simplify the following context from CEFR Level 3 to CEFR Level 1  7.954910   
Simplify the following context from CEFR Level 3 to CEFR Level 2  7.844883   

                                                                 model_response_fl  \
                                                                              mean   
instruction                                                                          
Simplify the following context from CEFR Level 2 to CEFR Level 1         81.643727   
Simplify the following context from CEFR Level 3 to CEFR Level 1         77.594250   
Simplify the following context from CEFR Level 3 to CEFR Level 2         77.294187   

                                                                            \
                                                                       std   
instruction                                                                  
Simplify the following context from CEFR Level 2 to CEFR Level 1  9.851194   
Simplify the following context from CEFR Level 3 to CEFR Level 1  9.337990   
Simplify the following context from CEFR Level 3 to CEFR Level 2  9.363398   

                                                                 response_dale  \
                                                                          mean   
instr

## CEFR preds

In [60]:
count_CEFR = df_new.groupby('instruction')['model_response_CEFR'].count()
print(count_CEFR)

oob_count_CEFR = df_new.groupby('instruction')['oob_model_response_CEFR'].count()
oob_count_CEFR

instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    381
Simplify the following context from CEFR Level 3 to CEFR Level 1    320
Simplify the following context from CEFR Level 3 to CEFR Level 2    320
Name: model_response_CEFR, dtype: int64


instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    381
Simplify the following context from CEFR Level 3 to CEFR Level 1    320
Simplify the following context from CEFR Level 3 to CEFR Level 2    320
Name: oob_model_response_CEFR, dtype: int64

## finetuned model response scores

In [22]:
avg_smog = df_new.groupby('instruction')['model_response_smog'].mean()
avg_flesch = df_new.groupby('instruction')['model_response_fl'].mean()
avg_dale = df_new.groupby('instruction')['model_response_dale'].mean()

sd_smog = df_new.groupby('instruction')['model_response_smog'].std()
sd_flesch = df_new.groupby('instruction')['model_response_fl'].std()
sd_dale = df_new.groupby('instruction')['model_response_dale'].std()

print("SMOG", avg_smog, sd_smog)
print("FLESCH", avg_flesch, sd_flesch)
print("DALE", avg_dale, sd_dale)

SMOG instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    7.853018
Simplify the following context from CEFR Level 3 to CEFR Level 1    8.478125
Simplify the following context from CEFR Level 3 to CEFR Level 2    8.556250
Name: model_response_smog, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    1.838026
Simplify the following context from CEFR Level 3 to CEFR Level 1    1.998312
Simplify the following context from CEFR Level 3 to CEFR Level 2    1.808191
Name: model_response_smog, dtype: float64
FLESCH instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    79.346457
Simplify the following context from CEFR Level 3 to CEFR Level 1    76.240625
Simplify the following context from CEFR Level 3 to CEFR Level 2    76.846875
Name: model_response_fl, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    9.661953
Simplify the following context from CEFR Le

## OOB model response scores

In [23]:
oob_avg_smog = df_new.groupby('instruction')['oob_model_response_smog'].mean()
oob_avg_flesch = df_new.groupby('instruction')['oob_model_response_fl'].mean()
oob_avg_dale = df_new.groupby('instruction')['oob_model_response_dale'].mean()

oob_sd_smog = df_new.groupby('instruction')['oob_model_response_smog'].std()
oob_sd_flesch = df_new.groupby('instruction')['oob_model_response_fl'].std()
oob_sd_dale = df_new.groupby('instruction')['oob_model_response_dale'].std()

print("SMOG", oob_avg_smog, oob_sd_smog)
print("FLESCH", oob_avg_flesch, oob_sd_flesch)
print("DALE", oob_avg_dale, oob_sd_dale)

SMOG instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    6.414698
Simplify the following context from CEFR Level 3 to CEFR Level 1    6.631250
Simplify the following context from CEFR Level 3 to CEFR Level 2    7.359375
Name: oob_model_response_smog, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    3.773341
Simplify the following context from CEFR Level 3 to CEFR Level 1    4.405926
Simplify the following context from CEFR Level 3 to CEFR Level 2    4.066713
Name: oob_model_response_smog, dtype: float64
FLESCH instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    73.482940
Simplify the following context from CEFR Level 3 to CEFR Level 1    77.306250
Simplify the following context from CEFR Level 3 to CEFR Level 2    75.965625
Name: oob_model_response_fl, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    128.615207
Simplify the following contex

## training data scores

In [24]:
train_avg_smog = df_new.groupby('instruction')['response_smog'].mean()
train_avg_flesch = df_new.groupby('instruction')['response_fl'].mean()
train_avg_dale = df_new.groupby('instruction')['response_dale'].mean()

train_sd_smog = df_new.groupby('instruction')['response_smog'].std()
train_sd_flesch = df_new.groupby('instruction')['response_fl'].std()
train_sd_dale = df_new.groupby('instruction')['response_dale'].std()

print("SMOG", train_avg_smog, train_sd_smog)
print("FLESCH", train_avg_flesch, train_sd_flesch)
print("DALE", train_avg_dale, train_sd_dale)

SMOG instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    7.349081
Simplify the following context from CEFR Level 3 to CEFR Level 1    7.425000
Simplify the following context from CEFR Level 3 to CEFR Level 2    8.100000
Name: response_smog, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    1.609652
Simplify the following context from CEFR Level 3 to CEFR Level 1    1.689169
Simplify the following context from CEFR Level 3 to CEFR Level 2    1.349329
Name: response_smog, dtype: float64
FLESCH instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    83.942257
Simplify the following context from CEFR Level 3 to CEFR Level 1    82.312500
Simplify the following context from CEFR Level 3 to CEFR Level 2    80.662500
Name: response_fl, dtype: float64 instruction
Simplify the following context from CEFR Level 2 to CEFR Level 1    8.753154
Simplify the following context from CEFR Level 3 to CEFR Leve